# Slit Width Trade

In [1]:
# stdlib
import math
from pathlib import Path

# external
import astropy.units as unit
from astropy.visualization import quantity_support
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect.systems.optical import spectrometers, sensors, foreoptics, masks, diffractors
from architect.libs import utillib
from architect import luts

## Setup
Run this section onwards when a parameter is updated.

### Parameters

In [2]:
# region constants

orbital_altitude = 550 * unit.km
target_wavelength = 1400 * unit.nm

#foreoptic
focal_length = 100 * unit.mm
foreoptic_diameter = 35 * unit.mm
image_diameter = 10 * unit.mm
incident_angle = 0 * unit.deg
spectral_order = 1

#diffractor
fringe_frequency = 587 * 1/unit.mm

#slit
slit_height = 7.68 * unit.mm
# endregion

# region variables
#slit_width = 1 * unit.mm
slit_width = np.arange(start=1, stop=9, step=1) * unit.mm

adjusted_slit_width, adjusted_slit_height = utillib.hypercast(slit_width, slit_height)
# endregion

<Quantity [[1.],
           [2.],
           [3.],
           [4.],
           [5.],
           [6.],
           [7.],
           [8.]] mm>

<Quantity [[7.68],
           [7.68],
           [7.68],
           [7.68],
           [7.68],
           [7.68],
           [7.68],
           [7.68]] mm>

### System Instantiation

In [3]:
# region components
sensor = sensors.TauSWIR()

foreoptic = foreoptics.Foreoptic(
    focal_length = focal_length, diameter = foreoptic_diameter, image_diameter = image_diameter
    )


diffractor = diffractors.VPHGrism(
    fringe_frequency = fringe_frequency
)

slit = masks.RectSlit(
    size = (adjusted_slit_width, adjusted_slit_height)
    )

payload = spectrometers.HyperspectralImager(
    sensor = sensor, foreoptic = foreoptic, slit = slit, diffractor = diffractor
    )

# endregion


# region systems
# endregion

(<Quantity [[1.],
            [2.],
            [3.],
            [4.],
            [5.],
            [6.],
            [7.],
            [8.]] mm>,
 <Quantity [[7.68],
            [7.68],
            [7.68],
            [7.68],
            [7.68],
            [7.68],
            [7.68],
            [7.68]] mm>)

## Pipeline
The computational graph

In [6]:
radiance = luts.load("atmosphere/radiance_min")

snr = payload.get_signal_to_noise(radiance = radiance, wavelength = target_wavelength).flatten()

bandpass = payload.get_bandpass(
    slit_width = slit_width, incident_angle = incident_angle, spectral_order = spectral_order
    )

display(snr)

<Quantity [ 4.59059663,  9.18029072, 13.76908267, 18.35697286, 22.94396172,
           27.53004962, 32.11523697, 36.69952417]>

'\nplt.plot(slit_width, snr_list)\nplt.show()\n'

## Plots

In [7]:
slit_width_label = f"Slit Width"
bandpass_label = f"Full Width at Half Maximum"
snr_label = f"SNR"
data = {
    slit_width_label: slit_width,
    bandpass_label: bandpass,
    snr_label: snr,
}

df = pd.DataFrame.from_dict(data=data)
fig = plot.line(
    df=df,
    x=slit_width_label,
    y=[
        bandpass_label,
        snr_label,
    ],
    title=None,
    dark=True,
)
fig.show()
display(df)

,Slit Width,Full Width at Half Maximum,SNR
0,1.0,5.87,4.590597
1,2.0,11.74,9.180291
2,3.0,17.61,13.769083
3,4.0,23.48,18.356973
4,5.0,29.35,22.943962
5,6.0,35.22,27.530050
6,7.0,41.09,32.115237
7,8.0,46.96,36.699524
